# OCHRE User Tutorial

OCHRE&trade; is a Python-based energy modeling tool designed to model flexible
end-use loads and distributed energy resources in residential buildings. OCHRE
includes detailed models for flexible devices including HVAC equipment, water
heaters, electric vehicles, solar PV, and batteries. It can examine the
impacts of novel control strategies on energy consumption and occupant comfort
metrics. OCHRE integrates with many of NREL's established modeling tools,
including [ResStock&trade;](https://resstock.nrel.gov/),
[BEopt&trade;](https://www.nrel.gov/buildings/beopt.html),
[EVI-Pro](https://www.nrel.gov/transportation/evi-pro.html),
[SAM](https://sam.nrel.gov/), and [HELICS](https://helics.org).

This tutorial covers the basics of how to install and run OCHRE, and provides
some examples for various use cases. It can be downloaded online
[here](https://github.com/NREL/OCHRE/blob/main/notebook/user_tutorial.ipynb).

More information about OCHRE can be found in [our
documentation](https://ochre-nrel.readthedocs.io/), on [NREL's
website](https://www.nrel.gov/grid/ochre.html), and from the [Powered By
OCHRE](https://www.youtube.com/watch?v=B5elLVtYDbI) webinar recording.

If you use OCHRE for your research or other projects, please fill out our
[user survey](https://forms.office.com/g/U4xYhaWEvs).

* [Set up](#setup)
  * [Running online](#online)
  * [Local installation](#install)
  * [Getting input files](#inputs)
* [Simulating a dwelling](#dwelling)
* [Simulating a single piece of equipment](#equipment)
  * [Electric vehicle](#ev)
  * [Water heater](#water_heater)
* [Simulating a fleet](#fleet)
  * [EV fleet](#ev-fleet)
  * [Multiple houses](#house-fleet)
* [Simulating with external controllers](#control)
  * [HVAC setpoint control](#hvac-control)
  * [EV managed charging](#ev-control)
  * [HPWH CTA-2045 control](#wh-control)
* [Links to other examples](#links)


## <a name="setup"></a>Set up


### <a name="online"></a>Running online

OCHRE can be run through an interactive Jupyter notebook. You may need to run
the code below and restart the notebook to install OCHRE.


In [ ]:
%pip install ochre-nrel

### <a name="install"></a>Local installation

OCHRE can be installed using `pip` from the command line with:

```
pip install ochre-nrel
```

Alternatively, you can install a specific branch, for example:

```
pip install git+https://github.com/NREL/OCHRE@dev
```

Note that OCHRE currently requires Python version >=3.9 and <3.12.

### <a name="inputs"></a>Getting input files

OCHRE `Dwelling` models typically require 3 inputs files:
* An HPXML file with building and equipment properties (.xml)
* An HPXML schedule file with occupant usage profiles (.csv)
* A weather file (.epw, or .csv with NSRDB format)

There are [multiple
ways](https://ochre-nrel.readthedocs.io/en/latest/InputsAndArguments.html#generating-input-files)
to generate or download OCHRE input files. This tutorial will use [sample
files](https://github.com/NREL/OCHRE/tree/main/ochre/defaults/Input%20Files)
from the ResStock 2024.2 dataset and from
[EnergyPlus](https://energyplus.net/weather), which are both [publicly
available](https://resstock.nrel.gov/datasets).


## <a name="dwelling"></a>Simulating a dwelling

OCHRE is most commonly used to model an entire residential dwelling, or house,
with many pieces of energy-consuming equipment. In addition to specifying
input files, OCHRE requires a simulation start time, time resolution, and
duration.

Many [additional
inputs](https://ochre-nrel.readthedocs.io/en/latest/InputsAndArguments.html#dwelling-arguments)
can be specified. For an example with more details, see
[bin/run_dwelling.py](https://github.com/NREL/OCHRE/tree/main/bin/run_dwelling.py).
There are also [command line and graphical user
interfaces](https://ochre-nrel.readthedocs.io/en/latest/Introduction.html#command-line-interface)
for simpler use cases.

The following code will initialize a Dwelling:


In [ ]:
import os
import datetime as dt

from ochre import Dwelling
from ochre.utils import default_input_path  # for using sample files

dwelling_args = {
    # Timing parameters
    "start_time": dt.datetime(2018, 1, 1, 0, 0),  # (year, month, day, hour, minute)
    "time_res": dt.timedelta(minutes=10),         # time resolution of the simulation
    "duration": dt.timedelta(days=1),             # duration of the simulation

    # Input files
    "hpxml_file": os.path.join(default_input_path, "Input Files", "bldg0112631-up11.xml"),
    "hpxml_schedule_file": os.path.join(default_input_path, "Input Files", "bldg0112631_schedule.csv"),
    "weather_file": os.path.join(default_input_path, "Weather", "USA_CO_Denver.Intl.AP.725650_TMY3.epw"),
}

# Create Dwelling model
dwelling = Dwelling(**dwelling_args)

The following code will simulate the dwelling. By default, it will return the
following results and save them to files:

- `df`: a Pandas DataFrame of time series results
- `metrics`: a dictionary of energy metrics
- `hourly`: a Pandas DataFrame of time series results with 1 hour resolution

In [ ]:
df, metrics, hourly = dwelling.simulate()

The time series results include the total house power, including electricity
real and reactive power and natural gas consumption. Many [additional
results](https://ochre-nrel.readthedocs.io/en/latest/Outputs.html) can be
saved by increasing the `verbosity` of the simulation.

In [3]:
df.head()

NameError: name 'df' is not defined

Metrics include the total house electricity and natural gas energy consumed. Many [additional
metrics](https://ochre-nrel.readthedocs.io/en/latest/Outputs.html#all-metrics) can be
saved by increasing the `metrics_verbosity` of the simulation.


In [ ]:
metrics

The `Analysis` module has useful data analysis functions for OCHRE output
data. The following code will reload the results and recalculate the metrics
from the previous run:


In [ ]:
import os
from ochre import Analysis

# Load results from previous run
sim_path = os.getcwd()
sim_name = "ochre"
df, metrics, hourly = Analysis.load_ochre(sim_path, sim_name)

# calculate metrics from the time series results
metrics2 = Analysis.calculate_metrics(df)


The `CreateFigures` module has useful visualization functions for OCHRE output
data. The following code will create a stacked plot of house power by end use:



In [ ]:
%matplotlib

from ochre import CreateFigures

# Plot results
fig = CreateFigures.plot_power_stack(df)

The following code will plot the average daily load profile:

In [ ]:
fig = CreateFigures.plot_daily_profile(df, 'Total Electric Power (kW)', plot_max=False, plot_min=False)

## <a name="equipment"></a>Simulating a single piece of equipment

OCHRE can simulate a single piece of equipment, including an electric vehicle,
water heater, solar PV system, or battery. Compared to simulating a full
dwelling, fewer input files and arguments are required. However, most
equipment require some [input
arguments](https://ochre-nrel.readthedocs.io/en/latest/InputsAndArguments.html#equipment-specific-arguments)
that often need to be specified manually.

This tutorial shows examples to simulate an EV and a water heater. For more
details and examples, see
[bin/run_equipment.py](https://github.com/NREL/OCHRE/blob/main/bin/run_equipment.py)

### <a name="ev"></a>Simulating an electric vehicle

The following code will initialize and then simulate an [electric
vehicle](https://ochre-nrel.readthedocs.io/en/latest/ModelingApproach.html#electric-vehicles).
The vehicle type (i.e., drive train), charging level, and range must be
specified.

In [ ]:
from ochre import ElectricVehicle

equipment_args = {
    "start_time": dt.datetime(2018, 1, 1, 0, 0),  # year, month, day, hour, minute
    "time_res": dt.timedelta(minutes=15),
    "duration": dt.timedelta(days=10),
    "verbosity": 3,
    "save_results": False,  # if True, must specify output_path
    # "output_path": os.getcwd(),

    # Equipment-specific parameters
    "vehicle_type": "BEV",
    "charging_level": "Level 1",
    "range": 200,
}

# Initialize equipment
equipment = ElectricVehicle(**equipment_args)

In [ ]:
# Simulate equipment
df = equipment.simulate()

df.head()


In [ ]:
fig = CreateFigures.plot_daily_profile(df, "EV Electric Power (kW)", plot_max=False, plot_min=False)


In [ ]:
fig = CreateFigures.plot_time_series_detailed((df["EV SOC (-)"],))


### <a name="water_heater"></a>Simulating a water heater

The following code will initialize and then simulate a [water
heater](https://ochre-nrel.readthedocs.io/en/latest/ModelingApproach.html#water-heating).
Multiple inputs are required, including the setpoint temperature, tank size,
and heat transfer coefficient. A time series schedule is required that
includes a water draw profile, the air temperature surrounding the water
heater ("Zone Temperature"), and the water mains temperature. 

In [ ]:
import numpy as np
import pandas as pd
from ochre import ElectricResistanceWaterHeater

# Create water draw schedule
start_time = dt.datetime(2018, 1, 1, 0, 0)  # year, month, day, hour, minute
time_res = dt.timedelta(minutes=1)
duration = dt.timedelta(days=10)
times = pd.date_range(
    start_time,
    start_time + duration,
    freq=time_res,
    inclusive="left",
)
water_draw_magnitude = 12  # L/min
withdraw_rate = np.random.choice([0, water_draw_magnitude], p=[0.99, 0.01], size=len(times))
schedule = pd.DataFrame(
    {
        "Water Heating (L/min)": withdraw_rate,
        "Zone Temperature (C)": 20,
        "Mains Temperature (C)": 7,
    },
    index=times,
)

equipment_args = {
    "start_time": start_time,  # year, month, day, hour, minute
    "time_res": time_res,
    "duration": duration,
    "verbosity": 3,
    "save_results": False,  # if True, must specify output_path
    # "output_path": os.getcwd(),
    # Equipment-specific parameters
    "Setpoint Temperature (C)": 51,
    "Tank Volume (L)": 250,
    "Tank Height (m)": 1.22,
    "UA (W/K)": 2.17,
    "schedule": schedule,
}

# Initialize equipment
wh = ElectricResistanceWaterHeater(**equipment_args)


In [ ]:
# Run simulation
df = wh.simulate()

# Show results
df.head()


In [ ]:
fig = CreateFigures.plot_daily_profile(
    df, "Water Heating Electric Power (kW)", plot_max=False, plot_min=False
)

In [ ]:
fig = CreateFigures.plot_time_series_detailed((df["Hot Water Outlet Temperature (C)"],))

## <a name="fleet"></a>Simulating a fleet

OCHRE can simulate multiple homes or pieces of equipment at once to model
a aggregation or a fleet of devices. The following examples show how to model
a fleet of EVs or a set of homes sequentially. See the [following
section](#control) for methods to run multiple simulations in parallel.

For more details and examples, see
[bin/run_fleet.py](https://github.com/NREL/OCHRE/blob/main/bin/run_fleet.py)
and
[bin/run_multiple.py](https://github.com/NREL/OCHRE/blob/main/bin/run_multiple.py)

### <a name="ev-fleet"></a>Simulating an EV fleet

The following code will set up a fleet of EV models, run each one
sequentially, and then plot the power of each EV.

In [ ]:
def setup_ev(i) -> ElectricVehicle:
    # randomly select vehicle type, range, and charging level
    vehicle_type = np.random.choice(["BEV", "PHEV"])
    charging_level = np.random.choice(["Level 1", "Level 2"])
    if vehicle_type == "BEV":
        range = round(np.random.uniform(100, 300))
    else:
        range = round(np.random.uniform(20, 70))

    # Option to specify a file with EV charging events
    # Defaults to older charging event data
    # equipment_event_file = None
    lvl = charging_level.lower().replace(" ", "_")
    equipment_event_file = os.path.join(default_input_path, "EV", f"{vehicle_type}_{lvl}.csv")

    # Initialize equipment
    return ElectricVehicle(
        name=f"EV_{i}",
        seed=i,  # used to randomize charging events. Not used for randomization above
        vehicle_type=vehicle_type,
        charging_level=charging_level,
        range=range,
        start_time=dt.datetime(2018, 1, 1, 0, 0),  # year, month, day, hour, minute
        time_res=dt.timedelta(minutes=15),
        duration=dt.timedelta(days=5),
        verbosity=1,
        save_results=False,  # if True, must specify output_path
        # output_path=os.getcwd(),
        equipment_event_file=equipment_event_file,
    )

# Create fleet
n = 4
fleet = [setup_ev(i + 1) for i in range(n)]


In [ ]:
def run_ev(ev: ElectricVehicle):
    df = ev.simulate()
    out = df["EV Electric Power (kW)"]
    out.name = ev.name
    return out

# Simulate fleet
results = []
for ev in fleet:
    results.append(run_ev(ev))

# combine load profiles
df = pd.concat(results, axis=1)

df.head()


In [ ]:
df.plot()


### <a name="house-fleet"></a>Simulating multiple houses

The following code will download two building models from the ResStock 2024.2
dataset and sequentially initialize and simulate them. 

In [ ]:
import shutil

from ochre import Analysis

# Note: see documentation for where to download other weather files
# https://ochre-nrel.readthedocs.io/en/latest/InputsAndArguments.html#weather-file
default_weather_file = os.path.join(default_input_path, "Weather", "G0800310.epw")

main_path = os.getcwd()

# Download ResStock files to current directory
buildings = ["bldg0112631"]
upgrades = ["up00", "up11"]
input_paths = []
for upgrade in upgrades:
    for building in buildings:
        input_path = os.path.join(main_path, building, upgrade)
        os.makedirs(input_path, exist_ok=True)
        Analysis.download_resstock_model(building, upgrade, input_path, overwrite=False)
        shutil.copy(default_weather_file, input_path)
        input_paths.append(input_path)


In [ ]:
from ochre.cli import create_dwelling

# Run Dwelling models sequentially
for input_path in input_paths:
    dwelling = create_dwelling(input_path, duration=7)
    dwelling.simulate()


## <a name="control"></a>Simulating with external controllers

OCHRE is designed to integrate with external controllers and other modeling
tools. External controllers can adjust the power consumption of any OCHRE
equipment using multiple [control
methods](https://ochre-nrel.readthedocs.io/en/latest/ControllerIntegration.html).
The following examples show device-level control methods for HVAC systems,
EVs, and water heaters.

For more details and examples, see
[bin/run_external_control.py](https://github.com/NREL/OCHRE/blob/main/bin/run_external_control.py).
There is also example code to run OCHRE in
[co-simulation](https://github.com/NREL/OCHRE/blob/main/bin/run_cosimulation.py)
using HELICS.

### <a name="hvac-control"></a>HVAC setpoint control

This control will reduce the heating setpoint by 1C from 5-9PM each day. It
adjusts the setpoint schedule before beginning the simulation; however, this
control can be achieved by setting the setpoint at every time step as well. We
use the same house model as the first example.

In [ ]:
# Update the simulation args to run at a finer time resolution
dwelling_args.update(
    {
        "time_res": dt.timedelta(minutes=1),  # time resolution of the simulation
        "duration": dt.timedelta(days=1),  # duration of the simulation
        "verbosity": 6,  # verbosity of time series files (0-9)
    }
)

# Initialize the Dwelling
dwelling = Dwelling(**dwelling_args)

# Get HVAC heater schedule
heater = dwelling.get_equipment_by_end_use("HVAC Heating")
schedule = heater.schedule

# Reduce heating setpoint by 1C from 5-9PM (setpoint is already in the schedule)
peak_times = (schedule.index.hour >= 17) & (schedule.index.hour < 21)
schedule.loc[peak_times, "HVAC Heating Setpoint (C)"] -= 1

# Adjust the HVAC deadband temperature (not in the schedule yet)
schedule["HVAC Heating Deadband (C)"] = 1
schedule.loc[peak_times, "HVAC Heating Deadband (C)"] = 2

# Reset the schedule to implement the changes
heater.reset_time()

# Simulate
df, _, _ = dwelling.simulate()


In [ ]:
cols_to_plot = [
    "HVAC Heating Setpoint (C)",
    "Temperature - Indoor (C)",
    "Temperature - Outdoor (C)",
    "Unmet HVAC Load (C)",
    "HVAC Heating Electric Power (kW)",
]
df.loc[:, cols_to_plot].plot()


### <a name="ev-control"></a>EV managed charging

This control implements "perfect" managed charging for an EV. At each time
step, it calculates the average power required to achieve 100% SOC by the end
of the parking session, and sets the charging power to that value.

In [ ]:
from ochre.Equipment.EV import EV_EFFICIENCY

equipment_args = {
    "start_time": dt.datetime(2018, 1, 1, 0, 0),  # year, month, day, hour, minute
    "time_res": dt.timedelta(minutes=60),
    "duration": dt.timedelta(days=20),
    "verbosity": 3,
    "save_results": False,  # if True, must specify output_path
    # "output_path": os.getcwd(),
    # Equipment parameters
    "vehicle_type": "BEV",
    "charging_level": "Level 1",
    "range": 150,
}

# Initialize
ev = ElectricVehicle(**equipment_args)

# slow charge from start to end of parking
for t in ev.sim_times:
    remaining_hours = (ev.event_end - t).total_seconds() / 3600
    remaining_kwh = (1 - ev.soc) * ev.capacity
    if t >= ev.event_start and remaining_hours:
        power = remaining_kwh / remaining_hours / EV_EFFICIENCY
        ev.update({"Max Power": power})
    else:
        ev.update()

df = ev.finalize()


In [ ]:
CreateFigures.plot_daily_profile(df, "EV Electric Power (kW)", plot_max=False, plot_min=False)


In [ ]:
df.loc[:, ["EV Electric Power (kW)", "EV Unmet Load (kWh)", "EV SOC (-)"]].plot()


### <a name="wh-control"></a>HPWH CTA-2045 control

This control implements the CTA-2045 Load Add and Load Shed control for a heat
pump water heater. The control will adjust the setpoint and deadband of the
water heater thermostat to increase load for 1 hour (at 7AM and 4PM) and then
reduce load for 1 hour (at 8AM and 5PM).

In [ ]:
from ochre import HeatPumpWaterHeater

# Define equipment and simulation parameters
setpoint_default = 51  # in C
deadband_default = 5.56  # in C
equipment_args = {
    "start_time": dt.datetime(2018, 1, 1, 0, 0),  # year, month, day, hour, minute
    "time_res": dt.timedelta(minutes=1),
    "duration": dt.timedelta(days=1),
    "verbosity": 6,  # required to get setpoint and deadband in results
    "save_results": False,  # if True, must specify output_path
    # "output_path": os.getcwd(),        # Equipment parameters
    "Setpoint Temperature (C)": setpoint_default,
    "Tank Volume (L)": 250,
    "Tank Height (m)": 1.22,
    "UA (W/K)": 2.17,
    "HPWH COP (-)": 4.5,
}

# Create water draw schedule
times = pd.date_range(
    equipment_args["start_time"],
    equipment_args["start_time"] + equipment_args["duration"],
    freq=equipment_args["time_res"],
    inclusive="left",
)
water_draw_magnitude = 12  # L/min
withdraw_rate = np.random.choice([0, water_draw_magnitude], p=[0.99, 0.01], size=len(times))
schedule = pd.DataFrame(
    {
        "Water Heating (L/min)": withdraw_rate,
        "Water Heating Setpoint (C)": setpoint_default,  # Setting so that it can reset
        "Water Heating Deadband (C)": deadband_default,  # Setting so that it can reset
        "Zone Temperature (C)": 20,
        "Zone Wet Bulb Temperature (C)": 15,  # Required for HPWH
        "Mains Temperature (C)": 7,
    },
    index=times,
)

# Initialize equipment
hpwh = HeatPumpWaterHeater(schedule=schedule, **equipment_args)

# Simulate
control_signal = {}
for t in hpwh.sim_times:
    # Change setpoint based on hour of day
    if t.hour in [7, 16]:
        # CTA-2045 Basic Load Add command
        control_signal = {"Deadband": deadband_default - 2.78}
    elif t.hour in [8, 17]:
        # CTA-2045 Load Shed command
        control_signal = {
            "Setpoint": setpoint_default - 5.56,
            "Deadband": deadband_default - 2.78,
        }
    else:
        control_signal = {}

    # Run with controls
    _ = hpwh.update(control_signal=control_signal)

df = hpwh.finalize()

df.head()


In [ ]:
cols_to_plot = [
    "Hot Water Outlet Temperature (C)",
    "Hot Water Average Temperature (C)",
    "Water Heating Deadband Upper Limit (C)",
    "Water Heating Deadband Lower Limit (C)",
    "Water Heating Electric Power (kW)",
    "Hot Water Unmet Demand (kW)",
    "Hot Water Delivered (L/min)",
]
df.loc[:, cols_to_plot].plot()
